# Language Modeling

## Imports & Inits

In [1]:
%load_ext autoreload
%autoreload 2
%config IPCompleter.greedy=True

import pdb, sys, warnings, os, json
warnings.filterwarnings(action='ignore')

from IPython.display import display, HTML
from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

np.set_printoptions(precision=4)
sns.set_style("darkgrid")
%matplotlib inline

## Data Setup

In [2]:
data_dir = Path('../data')

In [3]:
onion_df = pd.read_csv(data_dir/'onion_headlines.csv', usecols=['text', 'length'])
onion_df.shape

(14363, 2)

In [4]:
onion_df['length'].describe()

count    14363.000000
mean        78.252106
std         31.010523
min          1.000000
25%         57.000000
50%         75.000000
75%         95.000000
max        335.000000
Name: length, dtype: float64

In [5]:
onion_df.head()

,text,length
0,entire facebook staff laughs as man tightens p...,61
1,bold move: hulu has announced that they’re gon...,134
2,despondent jeff bezos realizes he’ll have to w...,101
3,"for men looking for great single women, online...",147
4,kim jong-un wonders if nuclear threats distrac...,91


## Checkpoint

In [8]:
vocab = list(set('\n'.join(onion_df['text'].tolist())))
len(vocab)

126